In [1]:
!nvidia-smi

Fri Nov 15 08:59:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import zipfile
import os

# Path
zip_path = '/content/archive (13).zip'
extract_path = '/content/CK+'

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check extracted folder structure
for root, dirs, files in os.walk(extract_path):
    print(root, len(files))


/content/CK+ 1
/content/CK+/fear 3176
/content/CK+/happy 5044
/content/CK+/neutral 5126
/content/CK+/disgust 2477
/content/CK+/sad 3091
/content/CK+/anger 3218
/content/CK+/contempt 2871
/content/CK+/surprise 4039


In [82]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import pickle

In [83]:
# Define paths and device
dataset_dir = '/content/CK+'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [84]:
# Load the dataset CSV file
df = pd.read_csv(os.path.join(dataset_dir, 'labels.csv'))  # Replace with your actual labels file name
df.drop(columns=['Unnamed: 0'], inplace=True)  # Drop unnecessary columns
print(df.head())

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])


                      pth     label    relFCs
0  anger/image0000006.jpg  surprise  0.873142
1  anger/image0000060.jpg     anger  0.852311
2  anger/image0000061.jpg     anger  0.800957
3  anger/image0000066.jpg   disgust  0.843079
4  anger/image0000106.jpg     anger  0.849108


In [85]:
# Save the mapping for future use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)



Label Mapping: {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprise': 7}


In [86]:
# Shuffle and split the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

In [87]:
# Custom Dataset Class
class CustomEmotionDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        label = self.dataframe.iloc[idx, 1]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)



In [88]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [89]:
# Create datasets and dataloaders
train_dataset = CustomEmotionDataset(train_df, dataset_dir, transform=transform)
test_dataset = CustomEmotionDataset(test_df, dataset_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [90]:
# Define the Model
class EmotionRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(EmotionRecognitionModel, self).__init__()
        self.features = models.resnet18(pretrained=False)  # Custom training, no pretraining
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        return self.features(x)

In [91]:
# Initialize the model, loss function, and optimizer
num_classes = len(label_mapping)
model = EmotionRecognitionModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [92]:
# Training Loop
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/30, Loss: 1.8474453829704447
Epoch 2/30, Loss: 1.3820134824049388
Epoch 3/30, Loss: 1.1645252639520254
Epoch 4/30, Loss: 1.0172473691879436
Epoch 5/30, Loss: 0.8929717774086811
Epoch 6/30, Loss: 0.7735964747185403
Epoch 7/30, Loss: 0.6509364063012685
Epoch 8/30, Loss: 0.5115610345458308
Epoch 9/30, Loss: 0.38075339076578196
Epoch 10/30, Loss: 0.25990485364558
Epoch 11/30, Loss: 0.18128914486431907
Epoch 12/30, Loss: 0.15879474459678358
Epoch 13/30, Loss: 0.1192825370296466
Epoch 14/30, Loss: 0.11125383447683977
Epoch 15/30, Loss: 0.09728397025102552
Epoch 16/30, Loss: 0.09305115873291296
Epoch 17/30, Loss: 0.08771569526222607
Epoch 18/30, Loss: 0.06804912625878046
Epoch 19/30, Loss: 0.06902413496818618
Epoch 20/30, Loss: 0.06681306384477421
Epoch 21/30, Loss: 0.06715560883699738
Epoch 22/30, Loss: 0.059288080200199565
Epoch 23/30, Loss: 0.05876117023649289
Epoch 24/30, Loss: 0.0583357449085947
Epoch 25/30, Loss: 0.04137915172548106
Epoch 26/30, Loss: 0.051004954350365866
Epoch 

In [ ]:
# Evaluate on Test Set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# Save the Model in .pkl Format
model_path = '/content/emotion_detection_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved to {model_path}!")
